# Data Validation


- **Need to install python(v3.9.21)**

### Installed packages are :

- anndata - 0.10.3
- jupyter - 1.1.1
- leidenalg - 0.10.1
- matplotlib - 3.9.4
- numpy - 1.26.4
- pandas - 2.2.3
- scanpy - 1.9.6
- seaborn - 0.13.2
- scikit-learn - 1.5.2

In [36]:
data = "pollen"
model = "gatgan"
# k = 90

In [ ]:
import scanpy as sc
import  numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import leidenalg
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys

/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
from matplotlib import rcParams

# --- Configure Matplotlib & environment ---
# Use a standard LaTeX-compatible font or any known font
rcParams['pdf.fonttype'] = 42  # Use TrueType fonts for editable text in illustrator
rcParams['ps.fonttype'] = 42   # Also for PostScript output
rcParams['font.family'] = 'DejaVu Sans' # A good default font
# %matplotlib inline # This is for Jupyter notebooks, not needed in a .py script

In [17]:
df = pd.read_csv("datafilt1.csv", na_values=['NA'])
df1 = pd.read_csv("datafilt2.csv", na_values=['NA'])
df_comb = pd.read_csv("datafilt_combined.csv", na_values=['NA'])
df.shape

(299, 100)

In [18]:
adata_gen = sc.AnnData(df)
adata_real = sc.AnnData(df1)
adata_comb = sc.AnnData(df_comb)


/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
# label = pd.read_csv(r"/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/real_data/pollenc.txt", header=None, names=["cell_type"])
# label

## FOR CBMC DATA
# label = pd.read_csv(r"/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/real_data/cell_type_cbmc.csv", header = 0, index_col = 0)

## FOR YAN DATA
label = pd.read_csv(r"/path/for/cell/type/data/pollenc.txt", header = None)
label.reset_index(drop=True, inplace=True)
label = label.rename(columns={label.columns[0]: "cell_type"})
label

In [ ]:
label.cell_type.value_counts()

In [21]:
label_encoder = LabelEncoder()     # transform categorical labels into integers

# add one new column into the df3
label['class_label_encoded'] = label_encoder.fit_transform(label['cell_type'])
y = label['class_label_encoded'].values

In [ ]:
set(y)

In [23]:
# from sklearn.preprocessing import LabelEncoder   
# label=pd.read_csv("/vol/eph/data/Data_for_cbmc/cbmcannot.csv")
# label = label.rename(columns={label.columns[0]: "cell_type"})
# class_labels = label.cell_type.value_counts()
# class_labels.values.tolist()
# label_encoder = LabelEncoder()     # transform categorical labels into integers


# # add one new column into the df3
# label['class_label_encoded'] = label_encoder.fit_transform(label['cell_type'])
# y = label['class_label_encoded'].values 

### SAME CODE AS ABOVE - SO DON'T NEEDED

## ARI of Gen Data

## ARI of Real Data

## combined